## Handling Unbalanced Datasets using Class Weights in Keras

We will be working with MNIST dataset to illustrate a simple strategy to handle unbalanced datasets.  

### Lets import the required libraries

In [65]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
import numpy as np

### Lets define a method to load the mnist dataset

In [66]:
def load_datasets():
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    return X_train, Y_train, X_test, Y_test

### define a method to normalize the image data

In [67]:
def normalize_data(trainX, testX):
    #reshape the data
    X_train = trainX.reshape((trainX.shape[0], 28, 28, 1))
    X_test = testX.reshape((testX.shape[0], 28, 28, 1))
    # converting to floats
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    # normalize in b2in 0,1
    X_train_norm = train_norm / 255.0
    X_test_norm = test_norm / 255.0
    
    return X_train_norm, X_test_norm

### Lets define a method to convert the integer label to one hot encoding

In [68]:
def transform_Y(Y_train, Y_test):
    Y_train_transform = to_categorical(Y_train)
    Y_test_transform = to_categorical(Y_test)
    
    return Y_train_transform, Y_test_transform

### Here we define a very simple CNN network for digit classification task

In [69]:
#lets define a simple CNN for mnist digit classification
def build_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(Flatten())
    
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation = 'softmax'))
    
    return model

### We use the weight computing method from Scikit-learn

In [70]:
def get_class_weights(Y_train):
    weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
    class_weights = {}
    
    for i in range(10):
        class_weights[i] = weights[i]
    
    return class_weights

### Lets do it now... :P

In [71]:
#loads the datasets
X_train, Y_train, X_test, Y_test = load_datasets()

#normalizing the x values
X_train_norm, X_test_norm = normalize_data(X_train, X_test)

#transorm the y values to one hot
Y_train_transform, Y_test_transform = transform_Y(Y_train, Y_test)

## Lets first train the model without class weight and test the model on test data

In [72]:
model_without_class_weight = build_model()

In [73]:
#compile the model
model_without_class_weight.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

In [74]:
#train the model without class weigths
history = model_without_class_weight.fit(X_train_norm, Y_train_transform, epochs = 10, batch_size = 56, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 22s 414us/sample - loss: 0.4136 - acc: 0.8681 - val_loss: 0.0640 - val_acc: 0.9805
Epoch 2/10
54000/54000 [==============================] - 26s 479us/sample - loss: 0.1289 - acc: 0.9618 - val_loss: 0.0473 - val_acc: 0.9843
Epoch 3/10
54000/54000 [==============================] - 28s 519us/sample - loss: 0.0987 - acc: 0.9717 - val_loss: 0.0415 - val_acc: 0.9870
Epoch 4/10
54000/54000 [==============================] - 36s 658us/sample - loss: 0.0895 - acc: 0.9746 - val_loss: 0.0361 - val_acc: 0.9902
Epoch 5/10
54000/54000 [==============================] - 38s 696us/sample - loss: 0.0846 - acc: 0.9764 - val_loss: 0.0381 - val_acc: 0.9885
Epoch 6/10
54000/54000 [==============================] - 38s 701us/sample - loss: 0.0846 - acc: 0.9772 - val_loss: 0.0341 - val_acc: 0.9917
Epoch 7/10
54000/54000 [==============================] - 37s 679us/sample - loss: 0.0878 - acc: 0.9778 -

### Lets test the model without class weight

In [75]:
model_without_class_weight.evaluate(X_test_norm, Y_test_transform)

10000/10000 [==============================] - 1s 128us/sample - loss: 0.2503 - acc: 0.9727


[0.25026279188394546, 0.9727]

## Lets train the model with class weight and test the model on test data again

In [76]:
model_with_class_weight = build_model()

In [77]:
model_with_class_weight.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

In [78]:
# get the class weights
class_weights = get_class_weights(Y_train)

In [79]:
#train the model without class weigths
history = model_with_class_weight.fit(X_train_norm, Y_train_transform, epochs = 10, batch_size = 56, validation_split=0.1, class_weight=class_weights)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 35s 639us/sample - loss: 0.3992 - acc: 0.8723 - val_loss: 0.0738 - val_acc: 0.9773
Epoch 2/10
54000/54000 [==============================] - 36s 659us/sample - loss: 0.1307 - acc: 0.9605 - val_loss: 0.0433 - val_acc: 0.9857
Epoch 3/10
54000/54000 [==============================] - 38s 713us/sample - loss: 0.0994 - acc: 0.9704 - val_loss: 0.0371 - val_acc: 0.9898
Epoch 4/10
54000/54000 [==============================] - 40s 736us/sample - loss: 0.0861 - acc: 0.9752 - val_loss: 0.0354 - val_acc: 0.9908
Epoch 5/10
54000/54000 [==============================] - 39s 727us/sample - loss: 0.0830 - acc: 0.9775 - val_loss: 0.0340 - val_acc: 0.9910
Epoch 6/10
54000/54000 [==============================] - 40s 739us/sample - loss: 0.0808 - acc: 0.9773 - val_loss: 0.0368 - val_acc: 0.9905
Epoch 7/10
54000/54000 [==============================] - 40s 741us/sample - loss: 0.0799 - acc: 0.9782 -

### Lets test the model with class weight

In [80]:
model_with_class_weight.evaluate(X_test_norm, Y_test_transform)

10000/10000 [==============================] - 1s 141us/sample - loss: 0.0434 - acc: 0.9866


[0.04339111114250263, 0.9866]

### See the improvement..  :D 

| Model Type | Test Accuracy |
|---------------------|----------------------|
| Model without class weight | 0.9727 |
| Model with class weight | 0.9866 |